# Part 2 - 3D mesh tallies

Mesh tallies can also be used to visualise neutron interactions spatially throughout a geometry in 3D.

This notebook allows users to create a simple geometry from a few different materials and plot the results of a 3D regular mesh tally applied to the geometry.

This first code block defines the model geometry, materials and neutron source.

In [1]:
import openmc
import os

# MATERIALS

breeder_material = openmc.Material(1, "PbLi")   # Pb84.2Li15.8
breeder_material.add_element('Pb', 84.2, percent_type='ao')
breeder_material.add_element('Li', 15.8, percent_type='ao', enrichment=7.0, enrichment_target='Li6', enrichment_type='ao')   # natural enrichment = 7% Li6
breeder_material.set_density('atom/b-cm', 3.2720171e-2)   # around 11 g/cm3

copper = openmc.Material(name='Copper')
copper.set_density('g/cm3', 8.5)
copper.add_element('Cu', 1.0)

eurofer = openmc.Material(name='iron')
eurofer.set_density('g/cm3', 7.75)
eurofer.add_element('Fe', 89.067, percent_type='wo')

mats = openmc.Materials([breeder_material, eurofer, copper])


# GEOMETRY

# surfaces
central_sol_surface = openmc.ZCylinder(r=100)
central_shield_outer_surface = openmc.ZCylinder(r=110)
vessel_inner_surface = openmc.Sphere(r=500)
first_wall_outer_surface = openmc.Sphere(r=510)
breeder_blanket_outer_surface = openmc.Sphere(r=610, boundary_type='vacuum')

# cells
central_sol_region = -central_sol_surface & -breeder_blanket_outer_surface
central_sol_cell = openmc.Cell(region=central_sol_region)
central_sol_cell.fill = copper

central_shield_region = +central_sol_surface & -central_shield_outer_surface & -breeder_blanket_outer_surface
central_shield_cell = openmc.Cell(region=central_shield_region)
central_shield_cell.fill = eurofer

inner_vessel_region = -vessel_inner_surface & +central_shield_outer_surface
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)
# no material set as default is vacuum

first_wall_region = -first_wall_outer_surface & +vessel_inner_surface
first_wall_cell = openmc.Cell(region=first_wall_region)
first_wall_cell.fill = eurofer

breeder_blanket_region = +first_wall_outer_surface & -breeder_blanket_outer_surface & +central_shield_outer_surface
breeder_blanket_cell = openmc.Cell(region=breeder_blanket_region)
breeder_blanket_cell.fill = breeder_material

universe = openmc.Universe(cells=[central_sol_cell, central_shield_cell, inner_vessel_cell, first_wall_cell, breeder_blanket_cell])
geom = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
batches = 2
sett.batches = batches
sett.inactive = 0
sett.particles = 5000
sett.run_mode = 'fixed source'

# Create a DT point source
source = openmc.Source()
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.space = openmc.stats.Point((150, 150, 0))

sett.source = source

This code block creates a 3D regular mesh between two coordinates with a specified resolution in each axis.

In [2]:
mesh = openmc.RegularMesh(mesh_id=1)  # note the mesh_id is set to 1, this is needed later

mesh.dimension = [100, 50, 100]  # number of mesh cells in each dimension (resolution)

mesh.lower_left = [-750, 0, -750]  # x,y,z coordinates
mesh.upper_right = [750, 750, 750]  # x,y,z coordinates

This code block creates two tallies on the mesh to record heating and tritium production.

In [3]:
tallies = openmc.Tallies()
# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)

# Create flux mesh tally to score flux
mesh_tally = openmc.Tally(tally_id=1, name='tbr_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['(n,Xt)']
tallies.append(mesh_tally)

# Create flux mesh tally to score flux
mesh_tally = openmc.Tally(tally_id=2, name='heating_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['heating']
tallies.append(mesh_tally)

This next code block performs the simulation.

In [4]:
# deletes old statepoint and summary files
!rm s*.h5

# Run OpenMC!
model = openmc.model.Model(geom, mats, sett, tallies)
sp_filename = model.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

This code block runs a python function which extracts the mesh tally data from the statepoint.h5 file and saves it as a vtk file.

In [6]:
from openmc_mesh_tally_to_vtk import write_mesh_tally_to_vtk


# loads up the output file from the simulation
statepoint = openmc.StatePoint('statepoint.2.h5')

# extracts the mesh tally by name
my_tally = statepoint.get_tally(name='tbr_on_mesh')

# converts the tally result into a VTK file
write_mesh_tally_to_vtk(
    tally=my_tally,
    filename = "tbr_tally_on_mesh.vtk",
)


# extracts the mesh tally by name
my_tally = statepoint.get_tally(name='heating_on_mesh')

# converts the tally result into a VTK file
write_mesh_tally_to_vtk(
    tally=my_tally,
    filename = "heating_tally_on_mesh.vtk",
)

tally.size 500000
tally [0. 0. 0. ... 0. 0. 0.]
Writing tbr_tally_on_mesh.vtk
tally.size 500000
tally [0. 0. 0. ... 0. 0. 0.]
Writing heating_tally_on_mesh.vtk


'heating_tally_on_mesh.vtk'

The next code block generates download links to the 3D vtk files created.

Click on the links to download the vtk files onto your base computer and open them with a VTK file reader such as Paraview or Visit.

Paraview can be downloaded here: https://www.paraview.org/download/.
Visit can be downloaded here: https://wci.llnl.gov/simulation/computer-codes/visit/downloads.

In [7]:
from IPython.display import FileLink
display(FileLink('heating_tally_on_mesh.vtk'))
display(FileLink('tbr_tally_on_mesh.vtk'))

/home/jshim/neutronics-workshop/tasks/task_08_CSG_mesh_tally/heating_tally_on_mesh.vtk

/home/jshim/neutronics-workshop/tasks/task_08_CSG_mesh_tally/tbr_tally_on_mesh.vtk

**Learning Outcomes for Part 2:**

- Mesh tallies can be used in neutronics simulations to measure a variety of different reactions such as neutron absorption, tritium production and flux.
- How neutrons are dissipated around the reactor.